In [ ]:
data_dir = ".\\data\\"

# SLC-1730, vac, 24hrs
#filename = "2018-07-26-14-39-24.tdms"
#keyence_filename= None

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename= None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from pprint import pprint
import numpy as np

from importlib import reload
reload(batch_utils)
reload(data_tools)

full_data, info = batch_utils.load_data(filename, keyence_filename, suffix="proc_data")

batch_utils.print_lengths(full_data, "Full dataset:")

limits = [full_data['set_point']['time'][0], full_data['set_point']['time'][-2]]
print("Using limits:")
pprint(limits)

data = dict(zip(full_data.keys(), [data_tools.truncate(limits, full_data[k]) for k in full_data]))
batch_utils.print_lengths(data, "Truncated dataset:")

batch_utils.print_structure(data)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

plotter = Scatter
#plotter = Scattergl

plots = [
    #plotter(x=data['position']['time'], y=np.ma.MaskedArray(data['position']['data_set_point_data'], 
    #                                                        mask=data['position']['mask_set_point']), name="Stage Error"),
    plotter(x=data['set_point']['time'], y=data['set_point']['avg_position_data_set_point_data'], name="Stage Error Average"),
    plotter(x=data['set_point']['time'], y=data['set_point']['max_position_data_set_point_data'], name="Stage Error Max"),
    plotter(x=data['set_point']['time'], y=data['set_point']['min_position_data_set_point_data'], name="Stage Error Min"),
    
    Histogram(y=data['set_point']['avg_position_data_set_point_data'], name="Stage Error Average", xaxis='x2'),

]

layout = Layout(
    width = 800,
    height = 800,
    title = filename,
    xaxis = dict(
        title = "Time",
        domain=[0, 0.7],
    ),
    yaxis = dict(
        title = "Position (mm)",
    ),
    xaxis2 = dict(
        title = "Log Prob",
        type = "log",
        domain=[0.75, 1],
    ),
    legend = dict(
        orientation="h",
    )
)

fig = Figure(data=plots, layout=layout)
iplot(fig, filename=filename)

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import copy

print("Joining up the data!")
slices = list(range(0, len(full_data['set_point']['time']), 1000))
print("Got {} slices".format(len(slices)))
#slices = slices[:40]

js = [min(j+100, len(full_data['set_point']['time'])) for j in slices[1:]]

merged_data = None
seams = dict(time=list(), label=list())

p = FloatProgress()
display(p)

p.max = len(js)

for i, j in zip(slices, js):
    print("\n\nGetting the next slice {} to {}".format(i, j))
    
    try:
        d, info = batch_utils.load_data(filename, keyence_filename, suffix="proc_data_part_{}-{}".format(i,j))
    except FileNotFoundError:
        print("Stopping at proc_data_part_{}-{}".format(i,j))
        break
    
    if merged_data is None:
        merged_data = copy.deepcopy(d)
    
    batch_utils.merge_data(merged_data, d)
    batch_utils.print_lengths(merged_data, "After merge:")
    
    seams['time'].append(d['set_point']['time'][0])
    seams['label'].append("i="+str(i))
    seams['time'].append(d['set_point']['time'][-1])
    seams['label'].append("j="+str(j))
    seams['time'].append(None)
    seams['label'].append(None)
         
    p.value += 1

print("Done!")

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

plotter = Scatter
#plotter = Scattergl

plots = [
    plotter(x=merged_data['set_point']['time'], y=merged_data['set_point']['avg_position_data_set_point_data'], name="Stage Error Average"),
    plotter(x=merged_data['set_point']['time'], y=merged_data['set_point']['max_position_data_set_point_data'], name="Stage Error Max"),
    plotter(x=merged_data['set_point']['time'], y=merged_data['set_point']['min_position_data_set_point_data'], name="Stage Error Min"),
    
    plotter(x=seams['time'], y=np.zeros(len(seams['time'])), text=seams['label'], name="Seams", mode='markers+lines'),
    
    Histogram(y=merged_data['set_point']['avg_position_data_set_point_data'], name="Stage Error Average", xaxis='x2'),

]

layout = Layout(
    width = 800,
    height = 800,
    title = filename,
    xaxis = dict(
        title = "Time",
        domain=[0, 0.7],
    ),
    yaxis = dict(
        title = "Position (mm)",
    ),
    xaxis2 = dict(
        title = "Log Prob",
        type = "log",
        domain=[0.75, 1],
    ),
    legend = dict(
        orientation="h",
    )
)

fig = Figure(data=plots, layout=layout)
iplot(fig, filename=filename)